In [1]:
import zipfile
import os


zip_file_path = r"C:\Users\Rutik\Desktop\BFL intership\archive.zip"
extract_folder = r"C:\Users\Rutik\Desktop\BFL intership\data" 


if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print("Extraction completed! Files saved in:", extract_folder)
else:
    print("Error: ZIP file not found at", zip_file_path)


Extraction completed! Files saved in: C:\Users\Rutik\Desktop\BFL intership\data


In [2]:
# contents of "Original Data"
original_data_path = os.path.join(extract_folder, "Original Data")
class_dirs = os.listdir(original_data_path)
class_dirs


['test', 'train']

In [3]:
# train class
train_path = os.path.join(original_data_path, "train")
train_classes = os.listdir(train_path)
train_classes


['Black Rot', 'ESCA', 'Healthy', 'Leaf Blight']

In [4]:
# inside train FILES
train_path = os.path.join(original_data_path, "train")
train_classes = os.listdir(train_path)
train_classes


['Black Rot', 'ESCA', 'Healthy', 'Leaf Blight']

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# size and batch
img_height, img_width = 224, 224
batch_size = 32

# Path
train_path =train_path = r"C:\Users\Rutik\Desktop\BFL intership\archive\Original Data\train"

test_path = r"C:\Users\Rutik\Desktop\BFL intership\archive\Original Data\test"

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Training generator
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Validation generator
val_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# Test generator
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)


Found 5779 images belonging to 4 classes.
Found 1443 images belonging to 4 classes.
Found 1805 images belonging to 4 classes.


In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load base model (without top)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#include_top=False: removing the default final dense layers
# Freeze base model layers: training faster and avoids overfitting
base_model.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x) # single 1D vector
x = Dropout(0.3)(x)#Randomly turns off 30% of neurons during training to prevent overfitting.
x = Dense(128, activation='relu')(x)#128 fully connected neural network 
x = Dropout(0.3)(x)
output = Dense(4, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1 (Conv2D)                │ (None, 112, 112, 32)      │             864 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bn_Conv1 (BatchNormalization) │ (None, 112, 112, 32)      │             128 │ Conv1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Conv1_relu (ReLU)             │ (None, 112, 112, 32)      │               0 │ bn_Conv1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 32)      │             288 │ Conv1_relu[0][0]           │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_BN    │ (None, 112, 112, 32)      │             128 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_relu  │ (None, 112, 112, 32)      │               0 │ expanded_conv_depthwise_B… │
│ (ReLU)                        │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             512 │ expanded_conv_depthwise_r… │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_BN      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand (Conv2D)       │ (None, 112, 112, 96)      │           1,536 │ expanded_conv_project_BN[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_BN             │ (None, 112, 112, 96)      │             384 │ block_1_expand[0][0]       │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_expand_relu (ReLU)    │ (None, 112, 112, 96)      │               0 │ block_1_expand_BN[0][0]    │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_pad (ZeroPadding2D)   │ (None, 113, 113, 96)      │               0 │ block_1_expand_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ block_1_depthwise             │ (None, 56, 56, 96)        │             864 │ block_1_pad[0][0]          │
│ (DepthwiseConv2D)             │                           │               

 Total params: 2,422,468 (9.24 MB)

 Trainable params: 164,484 (642.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [7]:
!pip install scipy


Defaulting to user installation because normal site-packages is not writeable
  Using cached scipy-1.16.1-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.16.1-cp313-cp313-win_amd64.whl (38.5 MB)


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Rutik\\AppData\\Roaming\\Python\\Python313\\site-packages\\scipy\\_lib\\_array_api_no_0d.py'
Check the permissions.


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10 
)


C:\Users\Rutik\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 401s 2s/step - accuracy: 0.5124 - loss: 1.1498 - val_accuracy: 0.9473 - val_loss: 0.2183
Epoch 2/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 292s 2s/step - accuracy: 0.8855 - loss: 0.3309 - val_accuracy: 0.9716 - val_loss: 0.1129
Epoch 3/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 536s 3s/step - accuracy: 0.9249 - loss: 0.2100 - val_accuracy: 0.9702 - val_loss: 0.0936
Epoch 4/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 347s 2s/step - accuracy: 0.9386 - loss: 0.1737 - val_accuracy: 0.9820 - val_loss: 0.0703
Epoch 5/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 327s 2s/step - accuracy: 0.9483 - loss: 0.1512 - val_accuracy: 0.9778 - val_loss: 0.0693
Epoch 6/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.9556 - loss: 0.1348 - val_accuracy: 0.9792 - val_loss: 0.0687
Epoch 7/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 309s 2s/step - accuracy: 0.9580 - loss: 0.1136 - val_accuracy: 0.9792 - val_loss: 0.0579
Epoch 8/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 311s 2s/step - accuracy: 0.9648 - loss: 0.0954 - val_accu

In [11]:
print("Training Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])


Training Accuracy: 0.9629693627357483
Validation Accuracy: 0.9826749563217163


In [12]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)


1805/1805 ━━━━━━━━━━━━━━━━━━━━ 32s 18ms/step - accuracy: 0.9695 - loss: 0.0888
Test Accuracy: 0.9800553917884827


In [13]:
model.save("disease_detector_model.h5")
print(" Model saved as disease_detector_model.h5")



 Model saved as disease_detector_model.h5


In [14]:
from tensorflow.keras.preprocessing import image
import numpy as np

img = image.load_img(r"C:\Users\Rutik\Desktop\BFL intership\image.JPG", target_size=(128, 128))  # Resize to match your model input


In [15]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model

# Load image
img_path = r"C:\Users\Rutik\Desktop\BFL intership\image.JPG"
img = image.load_img(img_path, target_size=(224, 224))  # MobileNetV2 input size

# Convert to array and normalize
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Load trained model
model = load_model("disease_detector_model.h5")

# Predict
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])

# Map class index to label
class_indices = train_generator.class_indices  # e.g., {'Black Rot': 0, 'ESCA': 1, ...}
class_labels = list(class_indices.keys())
predicted_label = class_labels[predicted_class]

print(f"Predicted Class: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 660ms/step
Predicted Class: Black Rot


In [17]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the trained model
model = load_model("disease_detector_model.h5")

# Class labels from your train_generator
class_indices = train_generator.class_indices
class_labels = list(class_indices.keys())

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    img = cv2.resize(frame, (224, 224))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    # Prediction
    pred = model.predict(img)
    class_index = np.argmax(pred[0])
    label = class_labels[class_index]
    confidence = np.max(pred[0]) * 100

    # Display label on frame
    cv2.putText(frame, f"{label} ({confidence:.2f}%)", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Grape Disease Detection", frame)

    # Break loop on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 590ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━

In [21]:
!pip install opencv-python


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 509.0 kB/s eta 0:01:17
    --------------------------------------- 0.5/39.5 MB 509.0 kB/s eta 0:01:17
    --------------------------------------- 0.8/39.5 MB 479.7 kB/s eta 0:01:21
   - -------------------------------------- 1.0/39.5 MB 629.1 kB/s eta 0:01:02
   - -------------------------------------- 

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Rutik\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Rutik\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\Rutik\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\Rutik\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "C:\Users\Rutik\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 466, in read
    s = self.fp.read(amt)
  File "C:\Users\Rutik\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705, in 